# EVALUACIÓN FINAL MÓDULO 2 
Este script realiza el proceso completo:
1. Fase 1: Extracción de 100 registros desde API y limpieza de DataFrame.
2. Fase 2: Creación de la base de datos 'adalab_pelis_json' mediante Python.
3. Fase 3: Inserción de datos.
4. Fase 4: Consultas.

In [ ]:
# Requisito previo: Instalación de librerías necesarias
# pip install requests / mysql-connector-python / pandas

import requests 
import mysql.connector 
import pandas as pd

## FASE 1: Extracción y transformación

In [ ]:
url_pelis_adalab = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

peticion = requests.get(url_pelis_adalab) # Petición a la API

In [ ]:
if peticion.status_code == 200: # Verificamos si la conexión ha sido exitosa (status 200 = ok)
        print("Conexión API exitosa.")
        datos_iniciales = peticion.json()
else: # Si algo falla, que nos devuelva el código para buscar la explicación del fallo
        print(f"Error al conectar con la API: {peticion.status_code}")

In [ ]:
if len(datos_iniciales) > 0: # Vemos cuántos registros tenemos y las claves del primero para saber qué podemos encontrar en la bbdd
        print(f"Registros totales detectados: {len(datos_iniciales)}")
        print("Claves del primer registro:", datos_iniciales[0].keys())


In [ ]:
# Aunque nos piden extraer 100 registros, que son los que hay, limitamos datos para evitar sobrecarga si la API crece
limite = 100 
datos_limitados = datos_iniciales[:limite] # Cogiendo del inicio al limite marcado

In [ ]:
display(datos_limitados) # Visualización rápida de los datos

In [ ]:
lista_peliculas = [] # Creamos una lista vacía y vamos añadiendo diccionarios completos (filas) en cada iteración.

for peli in datos_limitados: 
        nueva_peli = {
            'Titulo': peli.get('titulo', 'desconocido'), # Si no hay título, ponemos 'Desconocido' para evitar vacíos
            'Año': peli.get('año', 0), # Si no hay año, ponemos 0 para evitar vacíos
            'Duracion': peli.get('duracion', 0),
            'Genero': peli.get('genero', 'sin clasificar'),
            'Contenido adulto': peli.get('adultos', False), # Tipo de dato booleano
            'Subtitulos': peli.get('subtitulos',[])
         }
        lista_peliculas.append(nueva_peli)  # Añadimos la película procesada a la lista

In [ ]:
df_peliculas = pd.DataFrame(lista_peliculas)
print("DataFrame creado correctamente.")
pd.DataFrame(lista_peliculas) 

In [ ]:
# AJUSTES VISUALES : 
df_peliculas.index = range(1, len(df_peliculas) + 1) # Índice desde 1 (no desde 0)
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 1000)        # Ancho máximo para evitar saltos de línea
pd.set_option('display.colheader_justify', 'center') # Centrar encabezados
print(df_peliculas)
    

## FASE 2: Creación de la base de datos

In [ ]:
# Hacemos la conexión con el servidor
try:
# IMPORTANTE: Definir credenciales antes de ejecutar.
    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab', # Dejar password='' para entornos locales sin contraseña, en caso de error consultar si en el equipo tiene MySQL contraseña y ponerla
        host='127.0.0.1', # Nos conectamos al servidor de mysql, sin especificar la database porque la vamos a crear aquí
        auth_plugin = 'mysql_native_password'
    )
    print("¡Conexión exitosa a MySQL!")

except mysql.connector.Error as err:
    # Si es un error de acceso denegado (ej. contraseña o usuario incorrecto)
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña.")
    # Si la base de datos no existe
    # Para cualquier otro tipo de error
    else:
        print(err) 
        print("Código de Error:", err.errno) 
        print("SQLSTATE", err.sqlstate) 
        print("Mensaje", err.msg) 

In [ ]:
mycursor = cnx.cursor() 

mycursor.execute("CREATE SCHEMA IF NOT EXISTS pelis_adalab;") # Creamos la base de datos
mycursor.execute("USE pelis_adalab;") 


In [ ]:
# Para evitar duplicados y problemas en las tablas, dejamos un borrado de tablas para poder ejecutar todo el código correctamente
mycursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
mycursor.execute("DROP TABLE IF EXISTS peliculas_idiomas;")
mycursor.execute("DROP TABLE IF EXISTS peliculas;")
mycursor.execute("DROP TABLE IF EXISTS idiomas;")
mycursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

In [ ]:
mycursor = cnx.cursor() 

In [ ]:
# Creación de tablas: 1º tabla principal con las películas
mycursor.execute("USE pelis_adalab;") 
mycursor.execute("""
    CREATE TABLE  IF NOT EXISTS peliculas (
        id_pelicula INT AUTO_INCREMENT PRIMARY KEY,
        titulo VARCHAR(255) NOT NULL,
        fecha YEAR, 
        duracion INT,
        genero VARCHAR(100),
        adulto BOOLEAN
    );
""")
# En vez de año, utilizamos 'fecha' para evitar problemas con la ñ

In [ ]:
# Creación de 2º tabla: idiomas para poder realizar subconsultas y joins
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS idiomas (
        id_idioma INT AUTO_INCREMENT PRIMARY KEY,
        idioma VARCHAR(50) UNIQUE
    );
""")

In [ ]:
# Creación de la 3º tabla: con claves foráneas y la que establece referencias. Relación de n a m porque muchas películas pueden tener varios idiomas en sus subtitulos y vs.
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS peliculas_idiomas (
        id_pelicula INT,
        id_idioma INT,
        PRIMARY KEY (id_pelicula, id_idioma),
        FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula),
        FOREIGN KEY (id_idioma) REFERENCES idiomas(id_idioma)
    );
""")

## FASE 3: Inserción de datos

In [ ]:
mycursor.execute("USE pelis_adalab;") # Aseguramos trabajar en la bbdd pelis_adalab antes de insertar los datos
for contador, (index, fila) in enumerate (df_peliculas.iterrows()): # Recorreremos el DataFrame fila por fila y usamos enumerate para controlar el ID en lugar de depender del auto-increment que en versiones pasadas daba problemas
    id_actual_peli = contador + 1 #  Con un contador limpio para la generación de IDs empezando en 1
    titulo_peli = fila['Titulo'] # Preparamos los datos para la primera tabla en variables simples para que sea más fácil de leer
    fecha_peli = fila['Año']
    duracion_peli = fila['Duracion']
    genero_peli = fila['Genero']
    es_adulto = fila['Contenido adulto']
             
    mycursor.execute( 
        "INSERT INTO peliculas (id_pelicula, titulo, fecha, duracion, genero, adulto) VALUES (%s, %s, %s, %s, %s, %s)",
        (id_actual_peli, titulo_peli, fecha_peli, duracion_peli, genero_peli, es_adulto) # Insertamos los registros usando esas variables
    )

    lista_de_subtitulos = fila['Subtitulos'] # Sacamos la lista de subtítulos a una variable      
    # Como 'lista_de_subtitulos' ya es una lista ['es', 'en'], la recorremos con for
    for iso_code in lista_de_subtitulos:
    # Insertamos el idioma individualmente
        codigo_individual = str(iso_code).strip() # Buena práctica para limpiar posibles espacios
        if codigo_individual: # Si el idioma ya existe en la bbdd
            mycursor.execute("SELECT id_idioma FROM idiomas WHERE idioma = %s", (codigo_individual,))
            res_idioma = mycursor.fetchone() # Recuperamos el primer registro encontrado por el SELECT

            if res_idioma: # Si existe, recuperamos el ID existente
                id_actual_idioma = res_idioma[0]
            else: # Si no existe, lo insertamos
                mycursor.execute("INSERT INTO idiomas (idioma) VALUES (%s)", (codigo_individual,))
                id_actual_idioma = mycursor.lastrowid

                if not id_actual_idioma or id_actual_idioma == 0: # Buena prácitca: red de seguridad. Si .lastrowid falla en cargas masivas, hacemos verificación manual
                    mycursor.execute("SELECT id_idioma FROM idiomas WHERE idioma = %s", (codigo_individual,))
                    rescate = mycursor.fetchone()
                    if rescate:
                        id_actual_idioma = rescate[0]
                    else: 
                        print(f"ERROR CRÍTICO: No se pudo generar ID para {codigo_individual}")
                        continue
                
            mycursor.execute( # Con el id_idioma ya creado insertamos los datos a la tabla intermedia 'peliculas_idiomas'
                    "INSERT IGNORE INTO peliculas_idiomas (id_pelicula, id_idioma) VALUES (%s, %s)",
                    (id_actual_peli, id_actual_idioma)
                )
cnx.commit() # De vital importancia para que los cambios se apliquen a la bbdd
print("Datos guardados con éxito")
exito = True

In [ ]:
if cnx is not None and cnx.is_connected():
    cnx.close() # Cerramos conexión para la realización de la siguiente consulta

## FASE 4: Consultas

In [ ]:
# Hacemos la conexión con el servidor
# IMPORTANTE: Definir credenciales antes de ejecutar como en fase 2
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    database= 'pelis_adalab',
    auth_plugin = 'mysql_native_password'
)

### 1. ¿Cuántas películas tienen una duración superior a 120 minutos?

In [ ]:
mycursor = cnx.cursor() 
mycursor.execute("USE pelis_adalab;")

query1 = ("SELECT COUNT(*) as total FROM peliculas WHERE duracion > 120;")
mycursor.execute(query1)
resultado = mycursor.fetchall()
print(pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))


### 2. ¿Cuántas películas incluyen subtítulos en español?

In [ ]:
prequery2 = ("SELECT id_idioma FROM idiomas WHERE idioma = 'es';")
mycursor.execute(prequery2)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

In [ ]:
query2 = """
    SELECT COUNT(*) as total FROM peliculas AS p 
    JOIN peliculas_idiomas AS pi ON p.id_pelicula = pi.id_pelicula
    JOIN idiomas AS i ON pi.id_idioma = i.id_idioma
    WHERE i.id_idioma = 1"""
mycursor.execute(query2)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

### 3. ¿Cuántas películas tienen contenido adulto?

In [ ]:
query3 = ("SELECT COUNT(*) as total FROM peliculas WHERE adulto = True;") # Como es un dato que transforma en 0 y 1, podríamos poner 1 en vez de True y obtendríamos el mismo resultado
mycursor.execute(query3)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

### 4. ¿Cuál es la película más antigua registrada en la base de datos?

In [ ]:
query4_consulta_unica = ("SELECT titulo as la_mas_antigua FROM peliculas as peliculas ORDER BY fecha ASC LIMIT 1;")
mycursor.execute(query4_consulta_unica)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

In [ ]:
query4_subconsulta = """
    SELECT titulo, fecha FROM peliculas 
    WHERE fecha = (SELECT MIN(fecha) FROM peliculas)
"""
mycursor.execute(query4_subconsulta)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

### 5. Muestra el promedio de duración de las películas agrupado por género.

In [ ]:
query5 = ("SELECT genero, AVG(duracion) as promedio_min FROM peliculas GROUP BY genero ORDER BY promedio_min ASC;")
mycursor.execute(query5)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

### 6. ¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

In [ ]:
query6 = ("SELECT fecha, COUNT(*) as total FROM peliculas GROUP BY fecha ORDER BY total ASC")
mycursor.execute(query6)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

### 7. ¿Cuál es el año con más películas en la base de datos

In [ ]:
query7_consulta_unica = ("SELECT fecha as año_mas_pelis, COUNT(*) as total FROM peliculas GROUP BY año_mas_pelis ORDER BY total DESC LIMIT 1;")
mycursor.execute(query7_consulta_unica)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

In [ ]:
query7_subconsulta = """
    SELECT fecha as año_mas_pelis, COUNT(*) as total FROM peliculas 
    GROUP BY fecha 
    HAVING total = (
        SELECT MAX(conteo) FROM (
            SELECT COUNT(*) as conteo FROM peliculas GROUP BY fecha
            ) as tabla_conteos
    )
"""
mycursor.execute(query7_subconsulta)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

### 8. Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

In [ ]:
query8 = ("SELECT genero as genero, COUNT(*) as cantidad_peliculas FROM peliculas GROUP BY genero ORDER BY cantidad_peliculas DESC;")
mycursor.execute(query8)
resultado = mycursor.fetchall()
print( pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))

### 9. Muestra todas las películas cuyo título contenga la palabra "Godfather"

In [ ]:
query9 = ("SELECT titulo FROM peliculas WHERE titulo LIKE '%Godfather%'")
mycursor.execute(query9)
resultado = mycursor.fetchall()
if resultado:
    print(pd.DataFrame(resultado, columns=mycursor.column_names).to_string(index=False))
else:
    print("prueba con otra palabra, esta no está en la base de datos pelis_adalab")